In [1]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from itertools import combinations 
from numpy import savetxt
import random

2022-10-18 20:41:03.498195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-18 20:41:03.498245: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras

In [3]:
zero_input_df = pd.read_csv("data/input_data_nn_138.csv", sep=";")
zero_output_df = pd.read_csv("data/output_data_nn_138.csv", sep=";")


input_df = zero_input_df.loc[~(zero_input_df==0).all(axis=1)].reset_index()
output_df = zero_output_df.loc[~(zero_output_df==0).all(axis=1)].reset_index()
#print(output_df.describe().loc[['mean', 'std', 'min', 'max']].transpose().to_latex())

In [4]:
def augment(x, y, augmenter = None):
    x, augmenter = _add_interactions(x, augmenter)
    return x, y, augmenter, x.columns

def _add_interactions(df, augmenter):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]

    if augmenter:
        df = augmenter.transform(df)
    else:
        augmenter = PolynomialFeatures(interaction_only=True, include_bias=False)
        df = augmenter.fit_transform(df)

    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df, augmenter

In [5]:
def run_main(train_X, test_X, train_Y, test_Y, cur_fold, cur_run):
    data = [train_X, test_X, train_Y, test_Y]
    predicted_df = pd.DataFrame(columns=test_Y.columns, index=test_Y.index.values)
    new_df = test_Y.copy()

    cols_pred = ['tensao_barramento11']
    model_id = 'tensao_11'
    predicted_11 = run(new_df, cols_pred, data, cur_fold, cur_run)

    cols_pred = ['tensao_barramento12']
    model_id = 'tensao_12'
    predicted_12 = run(new_df, cols_pred, data, cur_fold, cur_run)

    cols_pred = ['tensao_barramento24']
    model_id = 'tensao_24'
    predicted_24 = run(new_df, cols_pred, data, cur_fold, cur_run)
    
    predicted_df['tensao_barramento11'] = predicted_11
    predicted_df['tensao_barramento12'] = predicted_12
    predicted_df['tensao_barramento24'] = predicted_24
        
    cols_pred = ['pot_ativa_inj_barramento11']
    model_id = 'pot_ativa_inj_barramento11'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_ativa_inj_barramento12']
    model_id = 'pot_ativa_inj_barramento12'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_ativa_inj_barramento24']
    model_id = 'pot_ativa_inj_barramento24'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions

    cols_pred = ['pot_reativa_inj_barramento11']
    model_id = 'pot_reativa_inj_barramento11'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento12']
    model_id = 'pot_reativa_inj_barramento12'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento24']
    model_id = 'pot_reativa_inj_barramento24'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions    

    predicted_df.to_csv(f'results-138/{cur_run}-{cur_fold}.csv')

In [6]:
def run(df, cols_pred, data, idx, cur_run):
    trainX, testX, trainY, testY = data

    trainY = trainY[cols_pred]
    testY = testY[cols_pred]

    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.transform(testX)
    
    print(f" ========== MODEL {cols_pred[0]} {idx} {cur_run} ============")
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)

    model = Sequential()
    model.add(Input(shape=(trainX.shape[1],)))
    model.add(Dense(trainX.shape[-1], activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(trainX.shape[-1], activation = 'relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(len(cols_pred)))

    model.compile(loss = 'mae', optimizer = 'adam')
    model.fit(
        trainX, trainY,
        batch_size = 30, epochs = 20,
        callbacks = [es],
        validation_data = (testX, testY),
        verbose=1
    )

    _predictions = model.predict(testX)
    rmse = sqrt(mean_squared_error(testY, _predictions))
    print(f"RMSE: {rmse}")
    
    return _predictions  

In [ ]:
for i in range(1, 6):
    cur_fold = 1
    kfold = KFold(n_splits=10, shuffle=True)

    X = input_df
    Y = output_df
    for train_ix, test_ix in kfold.split(input_df, output_df):
        train_X, test_X = X.loc[train_ix], X.loc[test_ix]
        train_y, test_y = Y.loc[train_ix], Y.loc[test_ix]

        run_main(train_X, test_X, train_y, test_y, cur_fold, i)

        cur_fold += 1

 ========== MODEL tensao_barramento11 1 1 ============


2022-10-18 20:41:10.415765: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-18 20:41:10.415823: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-18 20:41:10.415870: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (schlickmann): /proc/driver/nvidia/version does not exist
2022-10-18 20:41:10.416403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
7603/7621 [============================>.] - ETA: 0s - loss: 0.0285WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fe4d1335670> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 

7621/7621 [==============================] - 12s 2ms/step - loss: 0.0034 - val_loss: 0.0020
Epoch 15/20
7621/7621 [==============================] - 10s 1ms/step - loss: 0.0034 - val_loss: 0.0022
Epoch 16/20
7621/7621 [==============================] - 13s 2ms/step - loss: 0.0034 - val_loss: 0.0022
Epoch 17/20
7621/7621 [==============================] - 13s 2ms/step - loss: 0.0034 - val_loss: 0.0029
Epoch 18/20
7621/7621 [==============================] - 18s 2ms/step - loss: 0.0034 - val_loss: 0.0023
Epoch 19/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0033 - val_loss: 0.0023
Epoch 20/20
7621/7621 [==============================] - 10s 1ms/step - loss: 0.0034 - val_loss: 0.0024
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experiment

7621/7621 [==============================] - 10s 1ms/step - loss: 7.2367 - val_loss: 2.9145
Epoch 3/20
7621/7621 [==============================] - 10s 1ms/step - loss: 6.9783 - val_loss: 2.9986
Epoch 4/20
7621/7621 [==============================] - 10s 1ms/step - loss: 6.8512 - val_loss: 2.4358
Epoch 5/20
7621/7621 [==============================] - 10s 1ms/step - loss: 6.7071 - val_loss: 2.3590
Epoch 6/20
7621/7621 [==============================] - 10s 1ms/step - loss: 6.6470 - val_loss: 2.1688
Epoch 7/20
7621/7621 [==============================] - 10s 1ms/step - loss: 6.5472 - val_loss: 2.7953
Epoch 8/20
7621/7621 [==============================] - 10s 1ms/step - loss: 6.4902 - val_loss: 2.9290
Epoch 9/20
7621/7621 [==============================] - 10s 1ms/step - loss: 6.4246 - val_loss: 2.6350
Epoch 10/20
7621/7621 [==============================] - 10s 1ms/step - loss: 6.3525 - val_loss: 2.4024
Epoch 11/20
7621/7621 [==============================] - 10s 1ms/step - loss: 6.298

...

7621/7621 [==============================] - 9s 1ms/step - loss: 6.2504 - val_loss: 2.3857
Epoch 3/20
7621/7621 [==============================] - 9s 1ms/step - loss: 6.0050 - val_loss: 2.1690
Epoch 4/20
7621/7621 [==============================] - 9s 1ms/step - loss: 5.8610 - val_loss: 2.2816
Epoch 5/20
7621/7621 [==============================] - 9s 1ms/step - loss: 5.7432 - val_loss: 2.1690
Epoch 6/20
7621/7621 [==============================] - 9s 1ms/step - loss: 5.6589 - val_loss: 2.1599
Epoch 7/20
7621/7621 [==============================] - 9s 1ms/step - loss: 5.5972 - val_loss: 2.4728
Epoch 8/20
7621/7621 [==============================] - 9s 1ms/step - loss: 5.5104 - val_loss: 2.7981
Epoch 9/20
7621/7621 [==============================] - 9s 1ms/step - loss: 5.4460 - val_loss: 2.1745
Epoch 10/20
7621/7621 [==============================] - 9s 1ms/step - loss: 5.3980 - val_loss: 2.1030
Epoch 11/20
7621/7621 [==============================] - 9s 1ms/step - loss: 5.3188 - val_lo

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
7590/7621 [============================>.] - ETA: 0s - loss: 6.4947WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fe4cece3d30> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gas

Epoch 19/20
7621/7621 [==============================] - 9s 1ms/step - loss: 1.6472 - val_loss: 1.0211
Epoch 20/20
7621/7621 [==============================] - 9s 1ms/step - loss: 1.6382 - val_loss: 1.0442
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
794/794 [==============================] - 1s 649us/step
RMSE: 1.777268737574572
 ========== MODEL tensao_barramento11 9 2 ============
Epoch 1/20
Please report this to the TensorFlow team

7621/7621 [==============================] - 9s 1ms/step - loss: 0.0046 - val_loss: 0.0034
Epoch 3/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0040 - val_loss: 0.0027
Epoch 4/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0039 - val_loss: 0.0026
Epoch 5/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0038 - val_loss: 0.0023
Epoch 6/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0038 - val_loss: 0.0029
Epoch 7/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0037 - val_loss: 0.0026
Epoch 8/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0037 - val_loss: 0.0032
Epoch 9/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0036 - val_loss: 0.0024
Epoch 10/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0036 - val_loss: 0.0023
Epoch 11/20
7621/7621 [==============================] - 9s 1ms/step - loss: 0.0036 - val_lo

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
7618/7621 [============================>.] - ETA: 0s - loss: 10.2078WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fe4ced77700> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'ga

In [9]:
# indexes = np.array(df.index.tolist())
# random.shuffle(indexes)
# ids_to_pred = np.split(indexes, 3)

# results_all = []
# for i in range(4):
#     results_per_ids = []
#     for idx2 in range(len(ids_to_pred)):
#         results = []
#         run_main(df, i, idx2, ids_to_pred[idx2])
#         results_per_ids.append(results)
        
#     results_all.append(results_per_ids)

In [8]:
# for _result in results_all:
#     print(results_all.index(_result))
#     for fold in _result:
#         print(f"{_result.index(fold)}: {sum(fold)}")

In [26]:
print(f"Fold 0: {results_all[7][0]}")
print(f"Fold 1: {results_all[4][1]}")
print(f"Fold 2: {results_all[8][2]}")
print(f"Fold 3: {results_all[3][3]}")

Fold 0: [0.005618806104382909, 0.006374496128408189, 0.005003273984089787, 3.4735610445018055, 5.057355698378199, 3.6107973146928116, 7.466659649968084, 6.293739685672916, 2.4304372324265673]
Fold 1: [0.004912833436191945, 0.006315874222206748, 0.0050400543592956865, 5.882427740243566, 3.5072674248462787, 3.786373119903367, 6.751693446504084, 6.168161926473203, 2.822188923073245]
Fold 2: [0.004500589408310584, 0.004620877592182861, 0.005015842571451259, 3.953451113791048, 4.887246335988537, 4.617532544758241, 5.595597117037946, 5.241330146546544, 2.5569277207026158]
Fold 3: [0.005904257261356303, 0.004717165985727671, 0.005687405161361025, 3.2969980512393264, 5.985023419883928, 3.5217441168282497, 8.310837484354026, 6.692027891640731, 3.3616694706450936]


In [40]:
print(f"RMSE tensao_barramento11: {sqrt(mean_squared_error(df['tensao_barramento11'], df_best_predicted['tensao_barramento11']))}")
print(f"RMSE tensao_barramento12: {sqrt(mean_squared_error(df['tensao_barramento12'], df_best_predicted['tensao_barramento12']))}")
print(f"RMSE tensao_barramento24: {sqrt(mean_squared_error(df['tensao_barramento24'], df_best_predicted['tensao_barramento24']))}")

print(f"RMSE pot_ativa_inj_barramento11: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento11'], df_best_predicted['pot_ativa_inj_barramento11']))}")
print(f"RMSE pot_ativa_inj_barramento12: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento12'], df_best_predicted['pot_ativa_inj_barramento12']))}")
print(f"RMSE pot_ativa_inj_barramento24: {sqrt(mean_squared_error(df['pot_ativa_inj_barramento24'], df_best_predicted['pot_ativa_inj_barramento24']))}")


RMSE tensao_barramento11: 0.005263612667191903
RMSE tensao_barramento12: 0.005570651504552013
RMSE tensao_barramento24: 0.005194712520332189
RMSE pot_ativa_inj_barramento11: 0.0
RMSE pot_ativa_inj_barramento12: 0.0
RMSE pot_ativa_inj_barramento24: 0.0
